In [1]:
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset
import pandas as pd
import torch

image_dir = "/kaggle/input/celeba-attribute/img_align_celeba/img_align_celeba"
attr_path = "/kaggle/input/celeba-attribute/list_attr_celeba.txt"
split_path = "/kaggle/input/celeba-attribute/list_eval_partition.txt"

# Transforms for ResNet
transform = transforms.Compose([
    transforms.CenterCrop(178),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


In [2]:
attr_df = pd.read_csv(attr_path, sep='\s+', skiprows=1)
attr_df

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202595.jpg,-1,-1,1,-1,-1,-1,1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
202596.jpg,-1,-1,-1,-1,-1,1,1,-1,-1,1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
202597.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
202598.jpg,-1,1,1,-1,-1,-1,1,-1,1,-1,...,-1,1,-1,1,1,-1,1,-1,-1,1


In [3]:
# Make the index a column
attr_df.reset_index(inplace=True)
attr_df.rename(columns={"index": "image_id"}, inplace=True)

# Map -1 to 0 (for binary classification)
attr_df.replace(-1, 0, inplace=True)

# Load evaluation split
eval_df = pd.read_csv("/kaggle/input/celeba-attribute/list_eval_partition.txt", 
                      sep='\s+', header=None, names=["image_id", "split"])
eval_df


,image_id,split
0,000001.jpg,0
1,000002.jpg,0
2,000003.jpg,0
3,000004.jpg,0
4,000005.jpg,0
...,...,...
202594,202595.jpg,2
202595,202596.jpg,2
202596,202597.jpg,2
202597,202598.jpg,2


In [4]:
attr_df = attr_df.merge(eval_df, on="image_id")
attr_df

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,split
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,1,1,0,1,0,1,0,0,1,0
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,1,0
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,2
202595,202596.jpg,0,0,0,0,0,1,1,0,0,...,1,1,0,0,0,0,0,0,1,2
202596,202597.jpg,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,2
202597,202598.jpg,0,1,1,0,0,0,1,0,1,...,1,0,1,1,0,1,0,0,1,2


In [5]:
print(attr_df['Blond_Hair'].value_counts())


Blond_Hair
0    172616
1     29983
Name: count, dtype: int64


In [6]:
from PIL import Image

class CelebABinaryHairDataset(Dataset):
    def __init__(self, df, image_root, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_root = image_root
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_root, row['image_id'])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(row['Blond_Hair'], dtype=torch.float32)
        gender = torch.tensor(row['Male'], dtype=torch.int64)
        return image, label, gender


In [7]:
train_df = attr_df[attr_df['split'] == 0]
train_df['Blond_Hair'].value_counts()

Blond_Hair
0    138503
1     24267
Name: count, dtype: int64

In [8]:
# Get counts per (Blond_Hair, Male) pair
train_df = attr_df[attr_df['split'] == 0]

hair_gender_counts = train_df.groupby(['Blond_Hair', 'Male']).size().unstack(fill_value=0)
hair_gender_counts.columns = ['Female', 'Male']
hair_gender_counts.index.name = 'Blond_Hair'

print(hair_gender_counts)

            Female   Male
Blond_Hair               
0            71629  66874
1            22880   1387


In [9]:
def subclass_balanced_sample(df, n=None, seed=42):
    """
    Returns a balanced sample across (Blond_Hair, Male) subgroups.
    If n is None, uses the minimum available in any subgroup.
    """
    grouped = df.groupby(['Blond_Hair', 'Male'])
    min_count = min(grouped.size()) if n is None else n
    
    return grouped.apply(lambda x: x.sample(n=min_count, random_state=seed)).reset_index(drop=True)

balanced_train_df = subclass_balanced_sample(train_df, n=1387)

# Confirm distribution
print(balanced_train_df.groupby(['Blond_Hair', 'Male']).size())

Blond_Hair  Male
0           0       1387
            1       1387
1           0       1387
            1       1387
dtype: int64


/tmp/ipykernel_36/299869665.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return grouped.apply(lambda x: x.sample(n=min_count, random_state=seed)).reset_index(drop=True)


In [10]:
from torch.utils.data import DataLoader

# Split datasets
train_df = attr_df[attr_df['split'] == 0]
val_df   = attr_df[attr_df['split'] == 1]
test_df  = attr_df[attr_df['split'] == 2]

# Dataset instances
train_set = CelebABinaryHairDataset(train_df, image_dir, transform)
balanced_train_set = CelebABinaryHairDataset(balanced_train_df, image_dir, transform)
val_set   = CelebABinaryHairDataset(val_df, image_dir, transform)
test_set  = CelebABinaryHairDataset(test_df, image_dir, transform)

# Dataloaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=2)


In [17]:
import torch.nn as nn
import torchvision.models as models

# Load ResNet-18 pretrained
model = models.resnet18(pretrained=True)

# Replace the final layer for binary classification
model.fc = nn.Linear(model.fc.in_features, 2)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 208MB/s]


In [16]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)


In [11]:
from tqdm import tqdm

# Training function with tqdm
def train_epoch(model, dataloader, criterion, optimizer, device='cuda'):
    model.train()
    running_loss = 0.0

    for images, labels, gender in tqdm(dataloader, desc="Training", leave=False):
        images = images.to(device)
        labels = labels.long().to(device)  # required for CrossEntropyLoss
    
        optimizer.zero_grad()
        outputs = model(images)  # shape: (B, 2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Evaluation function
def evaluate(model, dataloader, device='cuda'):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total



In [12]:
def evaluate_groupwise(model, dataloader, device='cuda'):
    model.eval()
    group_correct = {
        "Blond Male": 0,
        "Blond Female": 0,
        "Non-Blond Male": 0,
        "Non-Blond Female": 0
    }
    group_total = {k: 0 for k in group_correct}

    with torch.no_grad():
        for images, labels, genders in tqdm(dataloader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu()
            labels = labels.cpu()
            genders = genders.cpu()

            for pred, label, gender in zip(preds, labels, genders):
                group = ("Blond " if label == 1 else "Non-Blond ") + ("Male" if gender == 1 else "Female")
                group_total[group] += 1
                if pred == label:
                    group_correct[group] += 1

    for group in group_correct:
        correct = group_correct[group]
        total = group_total[group]
        acc = 100 * correct / max(1, total)
        print(f"{group}: Accuracy = {acc:.2f}% ({correct}/{total})")

    overall_acc = 100 * sum(group_correct.values()) / max(1, sum(group_total.values()))
    print(f"\nOverall Accuracy = {overall_acc:.2f}%")
    return overall_acc


In [19]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device=device)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}")
    
    print(f"Evaluating on validation set...")
    val_acc = evaluate_groupwise(model, val_loader, device=device)
    print(f"Epoch {epoch+1}: Val Acc = {val_acc:.2f}%")

# Save the final model
torch.save(model.state_dict(), "resnet18_blond_classifier.pth")


Epoch 1: Train Loss = 0.2762
Evaluating on validation set...


Blond Male: Accuracy = 94.51% (172/182)
Blond Female: Accuracy = 89.53% (2573/2874)
Non-Blond Male: Accuracy = 91.00% (7531/8276)
Non-Blond Female: Accuracy = 90.65% (7737/8535)

Overall Accuracy = 90.67%
Epoch 1: Val Acc = 90.67%


Epoch 2: Train Loss = 0.0981
Evaluating on validation set...


Blond Male: Accuracy = 94.51% (172/182)
Blond Female: Accuracy = 95.58% (2747/2874)
Non-Blond Male: Accuracy = 90.88% (7521/8276)
Non-Blond Female: Accuracy = 84.22% (7188/8535)

Overall Accuracy = 88.73%
Epoch 2: Val Acc = 88.73%


Epoch 3: Train Loss = 0.0265
Evaluating on validation set...


Blond Male: Accuracy = 91.21% (166/182)
Blond Female: Accuracy = 91.68% (2635/2874)
Non-Blond Male: Accuracy = 93.43% (7732/8276)
Non-Blond Female: Accuracy = 89.85% (7669/8535)

Overall Accuracy = 91.62%
Epoch 3: Val Acc = 91.62%


Epoch 4: Train Loss = 0.0079
Evaluating on validation set...


Blond Male: Accuracy = 94.51% (172/182)
Blond Female: Accuracy = 94.08% (2704/2874)
Non-Blond Male: Accuracy = 90.10% (7457/8276)
Non-Blond Female: Accuracy = 87.60% (7477/8535)

Overall Accuracy = 89.65%
Epoch 4: Val Acc = 89.65%


Epoch 5: Train Loss = 0.0043
Evaluating on validation set...


Blond Male: Accuracy = 92.86% (169/182)
Blond Female: Accuracy = 92.66% (2663/2874)
Non-Blond Male: Accuracy = 92.65% (7668/8276)
Non-Blond Female: Accuracy = 89.31% (7623/8535)

Overall Accuracy = 91.22%
Epoch 5: Val Acc = 91.22%


In [28]:
model_test = models.resnet18()
model_test.fc = nn.Linear(model_test.fc.in_features, 2)  # 2 outputs for CE

checkpoint_path = "/kaggle/working/resnet18_blond_classifier.pth"
model_test.load_state_dict(torch.load(checkpoint_path, map_location='cuda'))

model_test = model_test.to('cuda')

evaluate_groupwise(model_test, test_loader, device='cuda')

Blond Male: Accuracy = 87.78% (158/180)
Blond Female: Accuracy = 92.70% (2299/2480)
Non-Blond Male: Accuracy = 92.20% (6947/7535)
Non-Blond Female: Accuracy = 90.79% (8867/9767)

Overall Accuracy = 91.53%


91.52890491934676

## CB Loss

In [14]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

class ClassBalancedLoss(nn.Module):
    def __init__(self, samples_per_cls, beta=0.9999):
        super().__init__()
        effective_num = 1.0 - np.power(beta, samples_per_cls)
        weights = (1.0 - beta) / np.array(effective_num)
        weights = weights / np.sum(weights) * len(samples_per_cls)
        self.class_weights = torch.FloatTensor(weights)

    def forward(self, logits, labels):
        if logits.device != self.class_weights.device:
            self.class_weights = self.class_weights.to(logits.device)
        return F.cross_entropy(logits, labels, weight=self.class_weights)


In [15]:
from torch.utils.data import WeightedRandomSampler

def get_subgroup_weights(genders, labels):
    # Subgroup counts from your data:
    subgroup_counts = {
        (0, 0): 71629,  # Non-Blond Female
        (1, 0): 66874,  # Non-Blond Male
        (0, 1): 22880,  # Blond Female
        (1, 1): 1387    # Blond Male
    }
    subgroup_weights = {k: 1.0 / v for k, v in subgroup_counts.items()}
    weights = [subgroup_weights[(int(g), int(l))] for g, l in zip(genders, labels)]
    return weights


In [18]:
model2 = models.resnet18(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Replace the final layer for binary classification
model2.fc = nn.Linear(model2.fc.in_features, 2)
model2.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
# # Extract labels and genders from your DataFrame
# train_labels = train_df['Blond_Hair'].values
# train_genders = train_df['Male'].values  # 1=Male, 0=Female

# # Get per-sample weights
# sample_weights = get_subgroup_weights(train_genders, train_labels)

# # Build the sampler
# sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# # Create DataLoader using sampler
# train_set = CelebABinaryHairDataset(train_df, image_dir, transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)



In [20]:
# Class counts: [Non-Blond, Blond]
samples_per_cls = [138503, 24267]
criterion = ClassBalancedLoss(samples_per_cls)

optimizer = torch.optim.Adam(model2.parameters(), lr=1e-4)


In [21]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_epoch(model2, train_loader, criterion, optimizer, device=device)
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}")
    
    print(f"Evaluating on validation set...")
    val_acc = evaluate_groupwise(model2, val_loader, device=device)
    print(f"Epoch {epoch+1}: Val Acc = {val_acc:.2f}%")

torch.save(model2.state_dict(), "resnet18_blond_classifier_full.pth")


Epoch 1: Train Loss = 0.1247
Evaluating on validation set...


Blond Male: Accuracy = 54.95% (100/182)
Blond Female: Accuracy = 91.48% (2629/2874)
Non-Blond Male: Accuracy = 99.25% (8214/8276)
Non-Blond Female: Accuracy = 93.92% (8016/8535)

Overall Accuracy = 95.43%
Epoch 1: Val Acc = 95.43%


Epoch 2: Train Loss = 0.1039
Evaluating on validation set...


Blond Male: Accuracy = 54.95% (100/182)
Blond Female: Accuracy = 91.30% (2624/2874)
Non-Blond Male: Accuracy = 99.20% (8210/8276)
Non-Blond Female: Accuracy = 94.14% (8035/8535)

Overall Accuracy = 95.48%
Epoch 2: Val Acc = 95.48%


Epoch 3: Train Loss = 0.0855
Evaluating on validation set...


Blond Male: Accuracy = 33.52% (61/182)
Blond Female: Accuracy = 85.49% (2457/2874)
Non-Blond Male: Accuracy = 99.71% (8252/8276)
Non-Blond Female: Accuracy = 96.53% (8239/8535)

Overall Accuracy = 95.68%
Epoch 3: Val Acc = 95.68%


KeyboardInterrupt: 

In [22]:
torch.save(model2.state_dict(), "resnet18_blond_classifier_full.pth")

## Focal Loss

In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class CBFocalLoss(nn.Module):
    def __init__(self, samples_per_cls, beta=0.9999, gamma=2.0):
        super().__init__()
        self.beta = beta
        self.gamma = gamma

        # Compute effective number of samples
        effective_num = 1.0 - np.power(self.beta, samples_per_cls)
        weights = (1.0 - self.beta) / np.array(effective_num)
        weights = weights / np.sum(weights) * len(samples_per_cls)
        self.class_weights = torch.FloatTensor(weights)  # shape: (num_classes,)

    def forward(self, logits, labels):
        if logits.device != self.class_weights.device:
            self.class_weights = self.class_weights.to(logits.device)

        # Convert labels to one-hot encoding
        labels_one_hot = F.one_hot(labels, num_classes=logits.size(1)).float()

        # Softmax for predicted class probabilities
        probs = F.softmax(logits, dim=1)
        probs = (probs * labels_one_hot).sum(dim=1)  # shape: (batch_size,)

        weights = self.class_weights[labels]  # shape: (batch_size,)

        # Focal loss component
        focal_modulation = (1 - probs) ** self.gamma
        loss = -weights * focal_modulation * torch.log(probs + 1e-12)

        return loss.mean()


In [26]:
import torch.optim as optim

# Model 3: ResNet-18 with CB Focal Loss
model3 = models.resnet18(pretrained=True)
model3.fc = nn.Linear(model3.fc.in_features, 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3 = model3.to(device)

samples_per_cls = [138503, 24267]  # [Non-Blond, Blond]
criterion3 = CBFocalLoss(samples_per_cls=samples_per_cls, beta=0.9999, gamma=2.0)

# Optimizer
optimizer3 = optim.Adam(model3.parameters(), lr=1e-4)


In [27]:
for epoch in range(num_epochs):
    train_loss = train_epoch(model3, train_loader, criterion3, optimizer3, device=device)
    print(f"Model 3 - Epoch {epoch+1}: Train Loss = {train_loss:.4f}")
    
    print(f"Evaluating Model 3 on validation set...")
    val_acc = evaluate_groupwise(model3, val_loader, device=device)
    print(f"Model 3 - Epoch {epoch+1}: Val Acc = {val_acc:.2f}%")


Model 3 - Epoch 1: Train Loss = 0.0329
Evaluating Model 3 on validation set...


Blond Male: Accuracy = 38.46% (70/182)
Blond Female: Accuracy = 84.17% (2419/2874)
Non-Blond Male: Accuracy = 99.64% (8246/8276)
Non-Blond Female: Accuracy = 96.68% (8252/8535)

Overall Accuracy = 95.57%
Model 3 - Epoch 1: Val Acc = 95.57%


Model 3 - Epoch 2: Train Loss = 0.0272
Evaluating Model 3 on validation set...


Blond Male: Accuracy = 46.70% (85/182)
Blond Female: Accuracy = 91.65% (2634/2874)
Non-Blond Male: Accuracy = 99.53% (8237/8276)
Non-Blond Female: Accuracy = 93.77% (8003/8535)

Overall Accuracy = 95.43%
Model 3 - Epoch 2: Val Acc = 95.43%


Model 3 - Epoch 3: Train Loss = 0.0234
Evaluating Model 3 on validation set...


Blond Male: Accuracy = 48.90% (89/182)
Blond Female: Accuracy = 90.47% (2600/2874)
Non-Blond Male: Accuracy = 99.40% (8226/8276)
Non-Blond Female: Accuracy = 94.08% (8030/8535)

Overall Accuracy = 95.36%
Model 3 - Epoch 3: Val Acc = 95.36%


Model 3 - Epoch 4: Train Loss = 0.0176
Evaluating Model 3 on validation set...


Blond Male: Accuracy = 29.67% (54/182)
Blond Female: Accuracy = 85.87% (2468/2874)
Non-Blond Male: Accuracy = 99.67% (8249/8276)
Non-Blond Female: Accuracy = 94.86% (8096/8535)

Overall Accuracy = 94.97%
Model 3 - Epoch 4: Val Acc = 94.97%


Model 3 - Epoch 5: Train Loss = 0.0113
Evaluating Model 3 on validation set...


Blond Male: Accuracy = 48.35% (88/182)
Blond Female: Accuracy = 86.92% (2498/2874)
Non-Blond Male: Accuracy = 99.40% (8226/8276)
Non-Blond Female: Accuracy = 94.80% (8091/8535)

Overall Accuracy = 95.15%
Model 3 - Epoch 5: Val Acc = 95.15%


In [28]:
torch.save(model3.state_dict(), "resnet18_blond_classifier_full_cb_focal.pth")

In [30]:
model_test = models.resnet18()
model_test.fc = nn.Linear(model_test.fc.in_features, 2)  # 2 outputs for CE

checkpoint_path = "/kaggle/working/resnet18_blond_classifier_full_cb_focal.pth"
model_test.load_state_dict(torch.load(checkpoint_path, map_location='cuda'))

model_test = model_test.to('cuda')

evaluate_groupwise(model_test, test_loader, device='cuda')

Blond Male: Accuracy = 53.33% (96/180)
Blond Female: Accuracy = 85.24% (2114/2480)
Non-Blond Male: Accuracy = 98.94% (7455/7535)
Non-Blond Female: Accuracy = 95.63% (9340/9767)

Overall Accuracy = 95.21%


95.2058911932672

## Stage Wise

In [33]:
model_4 = models.resnet18()
model_4.fc = nn.Linear(model_4.fc.in_features, 2)  # 2 outputs for CE

checkpoint_path = "/kaggle/input/balanced-data-model/pytorch/default/1/resnet18_blond_classifier balanced_dataset_1387.pth"
model_4.load_state_dict(torch.load(checkpoint_path, map_location='cuda'))

model_4 = model_4.to('cuda')

# Use full, imbalanced training dataframe
train_set = CelebABinaryHairDataset(train_df, image_dir, transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)

val_set = CelebABinaryHairDataset(val_df, image_dir, transform)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=4)

# Use standard CrossEntropyLoss (or CB loss if you wish, not focal)
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_4.parameters(), lr=1e-4)  # Lower LR for fine-tuning


In [34]:
num_epochs = 2
for epoch in range(num_epochs):
    train_loss = train_epoch(model_4, train_loader, criterion, optimizer, device='cuda')
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}")
    
    print("Evaluating on validation set...")
    val_acc = evaluate_groupwise(model_4, val_loader, device='cuda')
    print(f"Epoch {epoch+1}: Val Acc = {val_acc:.2f}%")


Epoch 1: Train Loss = 0.1196
Evaluating on validation set...


Blond Male: Accuracy = 47.25% (86/182)
Blond Female: Accuracy = 90.71% (2607/2874)
Non-Blond Male: Accuracy = 99.47% (8232/8276)
Non-Blond Female: Accuracy = 94.61% (8075/8535)

Overall Accuracy = 95.64%
Epoch 1: Val Acc = 95.64%


Epoch 2: Train Loss = 0.0979
Evaluating on validation set...


Blond Male: Accuracy = 51.65% (94/182)
Blond Female: Accuracy = 90.22% (2593/2874)
Non-Blond Male: Accuracy = 99.35% (8222/8276)
Non-Blond Female: Accuracy = 94.20% (8040/8535)

Overall Accuracy = 95.38%
Epoch 2: Val Acc = 95.38%


In [35]:
torch.save(model_4.state_dict(), "resnet18_blond_classifier_full_two_stage.pth")

In [36]:
model_test = models.resnet18()
model_test.fc = nn.Linear(model_test.fc.in_features, 2)  # 2 outputs for CE

checkpoint_path = "resnet18_blond_classifier_full_two_stage.pth"
model_test.load_state_dict(torch.load(checkpoint_path, map_location='cuda'))

model_test = model_test.to('cuda')

evaluate_groupwise(model_test, test_loader, device='cuda')

Blond Male: Accuracy = 54.44% (98/180)
Blond Female: Accuracy = 89.80% (2227/2480)
Non-Blond Male: Accuracy = 99.02% (7461/7535)
Non-Blond Female: Accuracy = 95.41% (9319/9767)

Overall Accuracy = 95.71%


95.70684300170323

## Frozen layers

In [37]:
import torch.optim as optim
import torch.nn as nn
from torchvision import models

# Load model
model_4 = models.resnet18()
model_4.fc = nn.Linear(model_4.fc.in_features, 2)  # Binary output

# Load pretrained checkpoint
checkpoint_path = "/kaggle/input/balanced-data-model/pytorch/default/1/resnet18_blond_classifier balanced_dataset_1387.pth"
model_4.load_state_dict(torch.load(checkpoint_path, map_location='cuda'))

# Freeze all layers first
for param in model_4.parameters():
    param.requires_grad = False

# Unfreeze layer4 and fc
for param in model_4.layer4.parameters():
    param.requires_grad = True
for param in model_4.fc.parameters():
    param.requires_grad = True

# Move model to GPU
model_4 = model_4.to('cuda')

# Dataset and loaders
train_set = CelebABinaryHairDataset(train_df, image_dir, transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)

val_set = CelebABinaryHairDataset(val_df, image_dir, transform)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=4)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_4.parameters()), lr=1e-4)


In [38]:
num_epochs = 2
for epoch in range(num_epochs):
    train_loss = train_epoch(model_4, train_loader, criterion, optimizer, device='cuda')
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}")
    
    print("Evaluating on validation set...")
    val_acc = evaluate_groupwise(model_4, val_loader, device='cuda')
    print(f"Epoch {epoch+1}: Val Acc = {val_acc:.2f}%")


Epoch 1: Train Loss = 0.1187
Evaluating on validation set...


Blond Male: Accuracy = 43.41% (79/182)
Blond Female: Accuracy = 87.47% (2514/2874)
Non-Blond Male: Accuracy = 99.47% (8232/8276)
Non-Blond Female: Accuracy = 95.37% (8140/8535)

Overall Accuracy = 95.46%
Epoch 1: Val Acc = 95.46%


Epoch 2: Train Loss = 0.0909
Evaluating on validation set...


Blond Male: Accuracy = 44.51% (81/182)
Blond Female: Accuracy = 86.74% (2493/2874)
Non-Blond Male: Accuracy = 99.47% (8232/8276)
Non-Blond Female: Accuracy = 95.33% (8136/8535)

Overall Accuracy = 95.34%
Epoch 2: Val Acc = 95.34%


In [39]:
torch.save(model_4.state_dict(), "resnet18_blond_classifier_full_two_stage_w_frozen.pth")

In [40]:
model_test = models.resnet18()
model_test.fc = nn.Linear(model_test.fc.in_features, 2)  # 2 outputs for CE

checkpoint_path = "resnet18_blond_classifier_full_two_stage_w_frozen.pth"
model_test.load_state_dict(torch.load(checkpoint_path, map_location='cuda'))

model_test = model_test.to('cuda')

evaluate_groupwise(model_test, test_loader, device='cuda')

Blond Male: Accuracy = 44.44% (80/180)
Blond Female: Accuracy = 85.44% (2119/2480)
Non-Blond Male: Accuracy = 99.27% (7480/7535)
Non-Blond Female: Accuracy = 96.41% (9416/9767)

Overall Accuracy = 95.66%


95.65674782085964

## Dreambooth

In [ ]:
!pip install diffusers transformers accelerate xformers
!pip install datasets
!pip install --upgrade bitsandbytes

In [ ]:
import pandas as pd

# Group name logic
def group_name(row):
    gender = 'male' if row['Male'] == 1 else 'female'
    hair = 'blond' if row['Blond_Hair'] == 1 else 'not_blond'
    return f'{gender}_{hair}'

attr_df['group'] = attr_df.apply(group_name, axis=1)

# Create a dictionary of 100 sampled images per group
sampled_df = attr_df.groupby('group').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)


In [ ]:
sampled_df['group'].value_counts()

In [ ]:
from huggingface_hub import login

# Replace this string with your actual HuggingFace token
HF_TOKEN = "hf_<your_token>"

login(token=HF_TOKEN)
